<h1> <span style="color:brown">L'algorithme de Flajolet et Martin</span></h1>
    
<h3> <span style="color:brown">Objectif</span>. On veut estimer le nombre de mots distincts d'un livre en utilisant très peu de mémoire. En considérant un livre comme étant une liste de mots, il s'agit donc d'estimer la cardinalité de cette liste.</h3>

Pour les tests, récupérer sur <https://www.gutenberg.org/> un livre du domaine public (i.e., l'auteur est mort, il y a au moins 70 ans) au format texte brut *UTF-8*.



La fonction **genereMots(nomFic)** ci-dessous est un générateur des mots du fichier **nomFic**.<br>
On utilise ce générateur pour définir le flux des données : une simple boucle **for mot in genereMots(nomFic): ...** permet d'énumérer tous les mots du fichier.

In [1]:
import string
from math import log, sqrt
import time
    
delimiteurs = string.punctuation + string.whitespace

def genereMots(nomFic):
    source = open(nomFic,'r')
    el = ''
    car = source.read(1)
    while car != '':
        if car in delimiteurs:
            if len(el) > 0:
                yield el
                el = ''
        else: el += car
        car = source.read(1)
    if len(el) > 0: yield el
    source.close()

 ---
 _Question 1_. Écrire une fonction **compteExact(flux)** qui détermine la cardinalité d'un **flux**.
 

In [3]:
# Test compteExact
flux = genereMots("Data/completeWorks_Shakespeare.txt")
cardExacte = compteExact(flux)
print("cardinalité : %d" %cardExacte)

cardinalité : 34957


Comme vu en cours, le calcul exact de la cardinalité nécessite un espace linéaire en la taille du fichier.
Maintenant, pour gagner sur l'espace, on va sacrifier l'exactitude et chercher une estimation aussi bonne que possible.

## <span style="color:brown">Première approche</span>
Dans un premier temps, on souhaite mettre en oeuvre la version élémentaire du comptage probabiliste.
On rappelle les grandes étapes :
<pre><code>
Initialiser un tableau <span style="color:blue">BITMAP</span> de <span style="color:blue">&#x2113;</span> zéros<br>
Pour chaque élément <span style="color:blue">mot</span> du flux faire
    <span style="color:blue">index = rho(hash(mot))</span>
    <span style="color:blue">BITMAP[index] = 1</span><br>
Calculer <span style="color:blue">mini</span> le plus petit <span style="color:blue">i</span> tel que <span style="color:blue">BITMAP[i] = 0</span><br>
Retourner <span style="color:blue">2<sup>mini</sup>/ &phi;</span>
</code></pre>

 ---
 _Question 2_. Écrire une fonction **rho** qui prend en entrée une valeur entière __val__ et retourne la position du premier **1** à droite dans la décomposition binaire de __val__

In [5]:
# Test rho
for val in (96,17):
    print('rho(%d) = %d' %(val,rho(val)))

rho(96) = 5
rho(17) = 0


---
_Question 3_. Écrire une fonction **calcBitmap** qui prend en entrée un __flux__ et, qui calcule et retourne le tableau **BITMAP**.<br>
(Pour la longueur **&#x2113;** du tableau, on prendra, par exemple, $24$. )

In [ ]:
# Test calcBitmap


---
_Question 4_. Écrire une fonction **signature** qui prend en entrée un tableau **bitmap** de bits et qui retourne sa signature, i.e., le plus petit indice __i__ 
tel que **bitmap[i] = 0**.

In [9]:
# Test signature
signature([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0])

14

---
_Question 5_. Écrire une fonction **premApproche** qui prend en entrée un __flux__ et qui retourne une **estimation** (élémentaire) de sa cardinalité.


In [10]:
Phi = .77351


In [ ]:
# Test
flux = genereMots("Data/completeWorks_Shakespeare.txt")
estimation = premApproche(flux)
print("cardinalité estimée : %.3f" %estimation)

---
_Question 6_.  Évaluer la qualité de l'estimation en calculant sa précision relative.

## <span style="color:brown">Deuxième approche</span>
Pour obtenir une meilleure estimation de la cardinalité, le principe est de diviser le flux des données en **m** lots (où **m** une puissance de __2__).

On rappelle les principales étapes de l'algorithme de Flajolet et Martin :
<pre><code>
    Initialiser une liste <span style="color:blue">BITMAPS</span> de <span style="color:blue">m</span> tables de <span style="color:blue">&#x2113;</span> zéros<br>
    Pour chaque élément <span style="color:blue">el</span> du flux faire
        <span style="color:blue">empreinte = hash(el)</span>
        <span style="color:blue">numFlux = empreinte %m</span>
        <span style="color:blue">index = rho(empreinte // m)</span>
        <span style="color:blue">BITMAPS[numFlux][index] = 1</span><br>
    <span style="color:blue">cumul = 0</span>
    Pour <span style="color:blue">numFlux</span> de <span style="color:blue">0</span> à <span style="color:blue">m-1</span> faire
        Calculer <span style="color:blue">mini</span> le plus petit <span style="color:blue">i</span> tq <span style="color:blue">BITMAPS[numFlux][i] = 0</span>
        <span style="color:blue">cumul = cumul + mini</span>
    <span style="color:blue">moy = cumul/m</span><br>
    Retourner <span style="color:blue">m &times; 2<sup>moy</sup>/&phi;</span>
 </code></pre>

---
_Question 7_. Écrire une fonction **calc_m_Bitmaps** qui prend en entrée un __flux__ et un entier **m** 
(puissance de __2__) et, qui calcule et retourne une liste de __m__ tableaux **BITMAP**.


In [ ]:
# test calc_m_Bitmaps


---
_Question 8_. Écrire une fonction **flajoletMartin** qui prend en entrée un flux et un entier **m** (puissance de __2__) et qui retourne une estimation de sa cardinalité.

In [ ]:
# test algoFlajoletMartin
flux = genereMots("Data/completeWorks_Shakespeare.txt")
estimationFM=algoFlajoletMartin(flux,2**4)
print("cardinalité estimée par l'algo de Flajolet et Martin : %.3f" %estimationFM)

---
_Question 9_. L'erreur standard est évaluée à $\frac{0.78}{\sqrt{m}}$. 
Est-ce que vos résultats confirment ce calcul ?

## <span style="color:brown">Comparaison de documents<span>
Pour comparer deux livres **A** et **B**, on définit leur <span style="color:green;font-weight: bold;">indice de similarité</span> comme le rapport entre le nombre des mots distincts qui sont communs aux deux et le nombre de mots distincts qui sont dans leur union :
$$sim(A,B) =\frac{|A \cap B|}{|A \cup B|}$$
On observe que le nombre des mots distincts communs à **A** et __B__ s'exprime comme le nombre des mots distincts de **A** plus le nombre des mots distincts de __B__ moins le nombre de mots distincts de leur union : **$|A \cap B| = |A| + |B| - |A \cup B|$**.



---
_Question 10_. Écrire une fonction **similarite** qui prend en entrée les flux associés à deux livres  et  __m__ une puissance de **2**, et qui calcule l'indice de similarité des deux livres.<br>
La difficulté est de faire en sorte de n'effectuer qu'une seule passe sur chaque livre.

In [ ]:
print(similarite("Data/Au_bonheur_des_dames_zola.utf8.txt","Data/laDameAuxCamelias_utf8.txt"))

---
_Question 11_. _Au bonheur des dames_ est-il plus proche de _La Dame aux Camélias_ ou de _La bête humaine_ ?<br>
_NB_. Supprimer les paragraphes en anglais qui se trouvent en tête et la fin des fichiers et qui peuvent biaiser les résultats.